In [1]:
import torch
import pandas as pd
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import DataLoader, TensorDataset 
BASE_DIR = "P:/pet ML"
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
X_tr = torch.load(f'{BASE_DIR}/data/tensor/X_tr.pt').to(device)
y_tr = torch.load(f'{BASE_DIR}/data/tensor/y_tr.pt').to(device)
X_test = torch.load(f'{BASE_DIR}/data/tensor/X_test.pt').to(device)
y_stats = torch.load(f'{BASE_DIR}/data/tensor/y_stats.pt')

In [3]:
input_feat = len(X_tr[0, :])

In [9]:
class model_nn(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(input_feat, 256)
        self.layer2 = nn.Linear(256, 128)
        self.layer3 = nn.Linear(128, 64)
        self.layer4 = nn.Linear(64, 1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.relu(x)

        x = self.layer2(x)
        x = self.relu(x)

        x = self.layer3(x)
        x = self.relu(x)

        x = self.layer4(x)
        return x

In [10]:
def rmsle(y_pred, y_true):
    return torch.sqrt(torch.mean((torch.log1p(y_pred) - torch.log1p(y_true)) ** 2))

In [18]:
model = model_nn().to(device)
opt = Adam(model.parameters(), lr=0.001)
EPOCH = 20
loss_fn = nn.MSELoss()
train_ds = TensorDataset(X_tr, y_tr)
train_loader = DataLoader(dataset=train_ds, batch_size=256, shuffle=True)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, patience=10, factor=0.5)

In [12]:
print(type(rmsle))
print(rmsle)

<class 'function'>
<function rmsle at 0x000001B24DC4B100>


In [19]:
model.train()
loss_list = []
for epoch in range(EPOCH):
    print(f'EPOCH {epoch+1} start')
    epoch_loss = 0
    for batch_X, batch_y in train_loader:
        opt.zero_grad()
        pred = model(batch_X)
        loss = loss_fn(pred.squeeze(), batch_y)
        loss.backward()
        opt.step()
        epoch_loss += loss.item()
        
    epoch_loss /= len(train_loader)
    loss_list.append(epoch_loss)
    scheduler.step(epoch_loss)
    print(f'loss {epoch+1}: {epoch_loss:.6f}, lr: {opt.param_groups[0]["lr"]:.5f}')
    if (len(loss_list)) > 2:
        if loss_list[-2] - loss_list[-1] < 0.000001:
            print(f'EARLY STOPPING HERE!!!')
            break

EPOCH 1 start
loss 1: 0.006386, lr: 0.00100
EPOCH 2 start
loss 2: 0.003613, lr: 0.00100
EPOCH 3 start
loss 3: 0.003530, lr: 0.00100
EPOCH 4 start
loss 4: 0.003468, lr: 0.00100
EPOCH 5 start
loss 5: 0.003439, lr: 0.00100
EPOCH 6 start
loss 6: 0.003412, lr: 0.00100
EPOCH 7 start
loss 7: 0.003388, lr: 0.00100
EPOCH 8 start
loss 8: 0.003365, lr: 0.00100
EPOCH 9 start
loss 9: 0.003360, lr: 0.00100
EPOCH 10 start
loss 10: 0.003339, lr: 0.00100
EPOCH 11 start
loss 11: 0.003348, lr: 0.00100
EARLY STOPPING HERE!!!


In [ ]:
model.eval()
pred_norm = model(X_test)
pred_real = pred_norm * y_stats['std'] + y_stats['mean']